# Sentiment Analysis using Naive Bayes
In this assignment, we will attempt to label tweets with sentiments (positive, neutral and negative) using Naive Bayes classifier, which has already been studied by you. Naive Bayes is a very basic approach to this problem, but gives surprisingly good accuracy sometimes. There are several elegant libraries for this problem, one of which will be briefly introduced in this notebook later. <br> 
<br> 
**Note:** Since Naive Bayes is a basic algorithm, a couple of very useful sklearn features have been incorporated in this assignment. They will help you write much more robust and clean code, and are applicable to any ML code you write. <br> 
References:
1. Pipeline: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
2. GridSearch: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

# Part 1 - Naive Bayes 

## 1. Importing required libraries - 2 Marks

In [74]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# import pipeline, CountVectorizer, TfidfTransformer, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV 

## 2. Reading dataset 

In [75]:
data=pd.read_csv('tweets.csv')
data.drop(data.columns[0],axis=1,inplace=True)
data.info()

#dropping null values
data.dropna(inplace=True)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380 entries, 0 to 1379
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweets  1375 non-null   object
 1   labels  1380 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 21.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1375 entries, 0 to 1379
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweets  1375 non-null   object
 1   labels  1375 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 32.2+ KB


,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


## 3. Text processing for the tweets [1+1= 2 Marks]

In [76]:
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing 
    
    # convert passed tweet to lower case 
    
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    
    
    tweet = tweet.lower()
    # use work_tokenize imported above to tokenize the tweet 
    tweet = word_tokenize(tweet)
    
    return [word for word in tweet if word not in stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shopclues/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/shopclues/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Process all tweets - 2 Marks

In [78]:
processed=[]

for tweet in data['tweets']:
    
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    cleaned = processTweet(tweet)
    if(len(cleaned) == 0):
        continue
    #print(cleaned)
    processed.append(' '.join(cleaned))

print(processed)

['obama called gop budget social darwinism nice try believe social creationism', 'teen years obama known use marijuana cocaine', 'ipa congratulates president barack obama leadership regarding jobs act washington apr 05 2012 business w ... url', 'rt at_user whatsromneyhiding connection supporters critical race theory .... oh wait obama romney ...', 'rt at_user obama approved targeted assassinations modern us prez read rt url', 'video shows federal officials joking cost lavish conference url obama crime p2 news tcot teaparty', "one chicago kid says `` obama man '' tells jesse watters gun violence chicago like `` world war 17 ''", "rt at_user american kid `` 're uk ohhh cool tea queen '' british kid `` like go mcdonalds obama", "valid explanation obama wo n't let women golf course whatsromneyhiding", 'president obama lt lindsay lohan rumors beginning cross shape lights st lt 1987 analyst64 dc bicycle courier lt video changes scramble', "obama 's gender advantage extends states 2012 decode

In [79]:
data['processed'] = processed

## 4. Create pipeline and define parameters for GridSearch

In [85]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

## 5. Split data into test and train [1 Mark]

In [86]:
# split data into train and test with split as 0.2 
X_train, X_test, Y_train, Y_test = train_test_split(data['processed'],data['labels'],test_size=0.2, random_state=42)

## 6. Perform classification (using GridSearch) - [3 Marks]

In [88]:
# perform GridSearch CV with 10 fold CV using pipeline and tuned_paramters defined above 

gsc = GridSearchCV(
        estimator=text_clf,
        param_grid=tuned_parameters,
        cv=10, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result = gsc.fit(X_train, Y_train)
best_params = grid_result.best_params_
print(best_params)

{'clf__alpha': 0.1, 'tfidf__norm': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


### Classification report - [2 Marks]

In [91]:
# print classification report after predicting on test set with best model obtained in GridSearch

y_true, y_pred = Y_test, gsc.predict(X_test)

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89       195
           1       0.69      0.66      0.67        61
           2       1.00      0.26      0.42        19

    accuracy                           0.82       275
   macro avg       0.85      0.62      0.66       275
weighted avg       0.83      0.82      0.81       275



## Important and interesting insight:

In [92]:
counts = data.labels.value_counts()
print(counts)

0    942
1    352
2     81
Name: labels, dtype: int64


We can see above that the class distribution is highly imbalanced, this would not lead to good sampling of the data for the classifier. For your learning, you could use SMOTE (https://imbalanced-learn.readthedocs.io/en/stable/api.html) to oversample the minority classes and then evaluate the performance with Naive Bayes and compare. 

# Part 2 - VADER sentiment analysis

**Valence Aware Dictionary and Sentiment Reasoner (VADER)** is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. VADER does not requires any training data but is constructed from a generalizable, valence-based, human-curated gold standard sentiment lexicon. (A sentiment lexicon is a list of lexical features e.g., words, which are generally labelled according to their semantic orientation as either positive or negative.). VADER has been found to be quite successful when dealing with social media texts, editorials, movie reviews, and product reviews. This is because VADER not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.

[Original Paper](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) <br> 
<br> 
( Install the library using `pip install vaderSentiment`)

In [96]:
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [97]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{} {}".format(sentence, str(score)))

Let's see how it performs on a custom sentence

In [98]:
sentiment_analyzer_scores("VADER is smart, handsome, and funny.")

VADER is smart, handsome, and funny. {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}


1. The Positive, Negative and Neutral scores represent the proportion of text that falls in these categories. This means our sentence was rated as 75% Positive, 25% Neutral and 0% Negative. Hence all these should add up to 1.

2. The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate. 

        positive sentiment: compound score >= 0.05
        neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
        negative sentiment: compound score <= -0.05

### 1. Punctuation

The use of an exclamation mark(!), increases the magnitude of the intensity without modifying the semantic orientation. For example, “The food here is good!” is more intense than “The food here is good.” and an increase in the number of (!), increases the magnitude accordingly.

In [99]:
# Baseline sentence
sentiment_analyzer_scores("The food here is good")

The food here is good {'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}


In [100]:
# Punctuation
print(sentiment_analyzer_scores("The food here is good!"))
print(sentiment_analyzer_scores("The food here is good!!"))
print(sentiment_analyzer_scores("The food here is good!!!"))

The food here is good! {'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.4926}
None
The food here is good!! {'neg': 0.0, 'neu': 0.534, 'pos': 0.466, 'compound': 0.5399}
None
The food here is good!!! {'neg': 0.0, 'neu': 0.514, 'pos': 0.486, 'compound': 0.5826}
None


### 2. Capitalization
Using upper case letters to emphasize a sentiment-relevant word in the presence of other non-capitalized words, increases the magnitude of the sentiment intensity. For example, “The food here is GREAT!” conveys more intensity than “The food here is great!”

In [101]:
# Baseline sentence
sentiment_analyzer_scores("The food here is great!")

The food here is great! {'neg': 0.0, 'neu': 0.477, 'pos': 0.523, 'compound': 0.6588}


In [102]:
# Capitalisation
sentiment_analyzer_scores("The food here is GREAT!")

The food here is GREAT! {'neg': 0.0, 'neu': 0.438, 'pos': 0.562, 'compound': 0.729}


### 3. Conjunctions
Use of conjunctions like “but”, signals a shift in sentiment polarity, with the sentiment of the text following the conjunction being dominant. “The food here is great, but the service is horrible” has mixed sentiment, with the latter half dictating the overall rating.

In [103]:
# Baseline sentence
sentiment_analyzer_scores("The food here is great")

The food here is great {'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'compound': 0.6249}


In [104]:
# Conjunctions
sentiment_analyzer_scores("The food here is great, but the service is horrible")

The food here is great, but the service is horrible {'neg': 0.31, 'neu': 0.523, 'pos': 0.167, 'compound': -0.4939}
